In [1]:
import pandas as pd
import requests
import json
import prettytable
from datetime import date

In [2]:
#----------------------------------------------- Query for Api ---------------------------------------------------
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SAF1', 'CUUR0000SA0E', 'CUUR0000SAM', 'CUUR0000SAH1'],"startyear":"2022", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
    
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.csv','w')
    output.write (x.get_string())
    output.close()

In [3]:
#-------------------------------------- Create Index DataFrame -------------------------------------------------
# Item Code Structure, CUUR0000 <- Signals from CPI report || SAF1 <- Exact Item. 
## To Do: Switch from code to series id 
Indexes={'code': ["CUUR0000SAF1", "CUUR0000SA0E", "CUUR0000SAM", "CUUR0000SAH1"], 
        'index': ["Food", "Energy", "Medical Care", "Shelter"],
        'weight': [.13372, .092, .08237, .32065]}
Indexes=pd.DataFrame(Indexes)

In [4]:
#------------------------------------- Creating List of CPIs ----------------------------------------------------
fps=["CUUR0000SAF1.csv", "CUUR0000SA0E.csv", "CUUR0000SAM.csv", "CUUR0000SAH1.csv"]

In [5]:
#-------------------------------------- Initializing Dataframes ------------------------------------------------
sdata={'year': [], 
        'period': [],
        'value': [],
       'index':[],
       'weight':[]}
sdata=pd.DataFrame(sdata)

In [6]:

for i in range(0,len(fps)):
#--------------------------------------- Reading in Outputed Files --------------------------------------------- 
    df=pd.read_csv(fps[i])
#----------------------------------------- Removing Unwanted Data -------------------------------------------- 
    df.columns = df.iloc[0]
    df = df.reindex(df.index.drop(0)).reset_index(drop=True)
    df.columns.name = None
    df=df.drop(df.index[0])
#----------------------------------------Creating Data Frame----------------------------------------------------------------
## Creating Column Headers
    cols=df.columns
    cols=cols[0]
    cols=cols.split("|")
    for j in range(0,len(cols)):
        cols[j]=cols[j].strip()
## Creating Rows
    rows=[]
    for j in range(0,len(df.index)):
        h=df.iloc[j]
        h=h[0]
        h=h.split("|")
        rows.append(h)
        for k in range(0, len(rows[j])):
            rows[j][k]=rows[j][k].strip()
    rows
## Convert to dataframe 
    CPI=pd.DataFrame(rows, columns=cols)
#------------------------------------------- Final clean of Data -----------------------------------------------
## Filtering Columns
    CPI=CPI[["series id", "year", "period", "value"]]
## Remove unnecessary rows
    CPI=CPI.drop(CPI.index[len(CPI)-1])
## Remove M from period
    CPI["period"]=CPI["period"].str[1:]
## Change Column Types 
    CPI[["year","period","value"]]=CPI[["year", "period", "value"]].apply(pd.to_numeric)
## Find Max Value
    max_x=CPI.nlargest(3, ['period'])
    max_x
#--------------------------------------- Merging Index values and weights with data   ------------------------- 
    merged=pd.merge(max_x, Indexes, how='left', left_on="series id", right_on="code")
    data2use=merged.drop(columns=["series id", "code"])
    data2use
#--------------------------------------- Appending Value to DataFrame    
    sdata=pd.concat([sdata,data2use])


In [7]:
#--------------------------------------  Append to Current Essential ------------------------------------------------
## Creating IndexValue Column
sdata['IndexValue']=sdata['weight']*sdata['value']
## Creating New Total Value Added by Index
ToAppend=sdata.groupby(by="period").sum()
## Cleaning New Month Essential CPI
ToAppend['year']=2022
ToAppend['Index']=ToAppend['IndexValue']
ToAppend=ToAppend.reset_index()
ToAppend=ToAppend[['period','year', 'Index']]
#ToAppend['M/M % Ch.']=(ToAppend['Index'] / ToAppend['Index'].shift(1) - 1).fillna(0)
## Import Essentials
EssentialCPI=pd.read_csv('CrudeEssential.csv')
currentMonth = str(date.today().month -2)
if (len(currentMonth)<2):
    currentMonth= "0"+currentMonth
    currentYear=str(date.today().year)[2:]
EssentialCPI=EssentialCPI.drop(columns=['Unnamed: 0'])
filename= currentMonth+currentYear+"CrudeEssential.csv"
EssentialCPI.to_csv(filename)
## Exporting Final Index Created
EssentialCPI=EssentialCPI.drop(EssentialCPI.nlargest(2,['period']).index.values)
EssentialCPI=pd.concat([EssentialCPI, ToAppend])
EssentialCPI=EssentialCPI.reset_index()
EssentialCPI=EssentialCPI.drop(columns=['index','Unnamed: 0','M/M % Ch.'])
EssentialCPI['M/M % Ch.'] = (EssentialCPI['Index'] / EssentialCPI['Index'].shift(1) - 1).fillna(0)
EssentialCPI['Annualized']=((1+EssentialCPI['M/M % Ch.'])**12)-1
EssentialCPI.to_csv('CrudeEssential.csv')





In [8]:
EssentialCPI

,period,year,Index,Annualized,M/M % Ch.
0,1.0,2022,216.656264,0.000000,0.000000
1,2.0,2022,218.512669,0.107808,0.008568
2,3.0,2022,222.584743,0.248031,0.018635
3,4.0,2022,223.718306,0.062854,0.005093
4,5.0,2022,226.711117,0.172885,0.013378
5,6.0,2022,230.366394,0.211590,0.016123
6,7.0,2022,230.271673,-0.004923,-0.000411
7,8.0,2022,229.807077,-0.023944,-0.002018
